<a href="https://colab.research.google.com/github/jeonggunlee/CUDATeaching/blob/master/hello_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPU 기반의 고성능 CUDA 프로그래밍**
## 한림대학교 소프트웨어 융합대학 이정근
## 2019년 2월

안녕하세요. 반갑습니다. 한림대학교 소프트웨어융합대학 이정근 교수라고 합니다.
이번 GPU CUDA 교육에 참여해 주셔서 감사합니다. 본 강의는 이론과 실습으로 이루어져 있으며, 실습은 Colab을 통해서 진행될 예정입니다.

우선 교육에 참여하신 모든 선생님들께서는 Github 및 Colab 계정을 만들어 주시면 감사드리겠습니다.

* Github 주소: www.github.com
* Colab 주소: colab.research.google.com
* Google Drive 연동 : https://github.com/jeonggunlee/CUDATeaching/blob/master/colab_gdrive.ipynb

*  *  *

Colab에서 CUDA Coding을 실습하기 위해서는 몇가지 사항을 알아야합니다.

* [코드 셀]에서 command-line 명령어 실행시키기
   - !ls : 현재 디렉토리의 내용을 보여준다.
   - %cd *dir*: *dir* 디렉토리로 이동한다.
   - %pwd: 현재 위치한 디렉토리 위치를 보여준다.
   - !git: git 명령어를 실행시킨다.

* 또한 GPU 장치와 관련되어 다음 명령어를 확인해주시기 바랍니다.
   - !nvidia-smi: 현재 사용하고 있는 GPU의 스펙과 작동 상황을 보여준다.
   - !nvcc: Nvidia CUDA Compiler를 실행시킨다.
   
위의 명령어들을 실행 시켜 보시기 바랍니다. 

아래의 내용은 youtube 동영상을 통하여 확인할 수 있습니다. [colab을 이용한 GPU CUDA 프로그래밍 ](https://youtu.be/pT38R3jXwe0)


처음 명령어로 pwd 명령어를 실행시켜보도록 하겠습니다. pwd는 사용자가 위치한 디렉토리를 보여줍니다.

In [1]:
!pwd

/content


현재의 디렉토리가 content에 위치하고 있네요. 기본적으로 colab을 사용하는 사용자는 항상 위와 같은 위치를 초기 디렉토리로 사용하게 됩니다.
다음으로 Unix/Linux 계열의 가장 기본적인 명령인 ls를 사용해보도록 하겠습니다. ls 명령어는 현재 디렉토리에 있는 화일 또는 디렉토리를 보여줍니다.

In [2]:
!ls

sample_data


위에서 보는 바와 같이 sample_data라는 디렉토리가 하위 디렉토리에 있다는 것을 알 수 있습니다.

다음으로 우리가 사용하고 있는 시스템의 프로세서 및 GPU에 대해서 알아보도록 하지요.
우선 CPU에 대한 정보를 얻기 위해서 Proc file 시스템의 cpuinfo를 cat 명령어로 보도록 해보지요.

In [3]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power ma

메모리 및 디스크 사용량을 알아볼까요 ?

1.   메모리 사용량: !cat /proc/meminfo
2.   디스크 사용량: !df -h



In [4]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G   20G  321G   6% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G  8.0K  6.4G   1% /var/colab
/dev/sda1       365G   24G  342G   7% /opt/bin
shm             6.0G     0  6.0G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware


자 다음엔 GPU의 스펙을 살펴보도록 하지요. GPU에 대한 정보를 얻기 위해서는 nvidia-smi 명령어를 사용합니다.

In [5]:
!nvidia-smi

Wed Jan 30 02:51:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

CPU와 GPU에 대한 정보를 확인 한 후에는 우리가 수행하고자 하는 코드들을 github로 부터 가져와서 시행해보도록 하겠습니다.

저같은 경우는 https://github.com/jeonggunlee/CUDATeaching 의 내용을 git 명령어를 통하여 clone 해오도록 하겠습니다.


In [6]:
!git clone https://github.com/jeonggunlee/CUDATeaching

Cloning into 'CUDATeaching'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 192 (delta 30), reused 0 (delta 0), pack-reused 139
Receiving objects: 100% (192/192), 21.18 MiB | 31.39 MiB/s, done.
Resolving deltas: 100% (80/80), done.


Git clone이 완료된 후에 올바로 clone이 되었는지 확인하기 위하여 ls 명령어를 수행해보독 하겠습니다.

In [7]:
!ls

CUDATeaching  sample_data


이후, cd 명령어를 이용하여 CUDATeaching 디렉토리로 들어가보도록 하겠습니다. 주의 할점은 시스템 명령어를 시행할때 ! 또는 %를 붙여주어야 한다는 것입니다.

In [8]:
%cd CUDATeaching

/content/CUDATeaching


다시 CUDATeaching 디렉토리로 들어온 후에 ls 명령어를 통하여 디렉토리에 어떤 화일이 있는지 살펴보도록 하겠습니다.

In [9]:
!ls

'00_병렬 컴퓨팅 소개.pdf'   colab_gdrive.ipynb	 hello_CUDA.ipynb
 01_cuda_lab		    gpu4cloud.md	 README.md


위 화일 및 디렉토리 중,01_cuda_lab에 들어가 보도록 하겠습니다. 01_cuda_lab에는 실습을 위한 cuda 소스 코드들이 있습니다.

In [11]:
%cd 01_cuda_lab

/content/CUDATeaching/01_cuda_lab


01_cuda_lab에 들어온후에 ls 명령어를 통하여 어떤 화일들이 있는지 확인해보록 하겠습니다.

In [12]:
!ls

00_reduction_all.cu	     06_matmul_tile3_noBankConflict.cu	bankConflict.cu
00_reduction_final.cu	     07_matmul_tile4_unroll.cu		clock.cu
01_reduction.cu		     1hello.cu				CUDA_lab.txt
02_reduction.cu		     2Dindex.cu				globalMemory.cu
03_matmul.cu		     2index.cu				README.md
04_matmul_tile.cu	     async_streams.cu			transpose.cu
05_matmul_tile2_mem_coel.cu  atomicAdd.cu


가장 기본 적인 코드로 1hello.cu 를 컴파일 하고 실행시켜보도록 하겠습니다.
1hello.cu의 내용은 아래와 같습니다.

```cuda
#include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  helloCUDA<<<3, 4>>>();
  cudaDeviceReset();
  return 0;
}
```


1hello.cu를 컴파일하기 위하여 nvdia GPU 컴파일러 명령어인 nvcc를 사용합니다.
nvcc 컴파일러가 어디에 위치하는지 먼저 살펴볼까요 ?

In [13]:
!which nvcc

/usr/local/cuda/bin/nvcc


위의 명령어를 통하여 nvcc 컴파일러가 /usr/local/cuda/bin/에 위치하고 있음을 알 수 있습니다.
다음은 nvidia GPU 카드가 설치 되었는지 확인해보는 작업을 진행해보겠습니다.


In [14]:
!ls -l /dev/nv*

crw-rw-rw- 1 root root 195,   0 Jan 30 02:41 /dev/nvidia0
crw-rw-rw- 1 root root 195, 255 Jan 30 02:41 /dev/nvidiactl
crw-rw-rw- 1 root root 248,   0 Jan 30 02:41 /dev/nvidia-uvm
crw-rw-rw- 1 root root 248,   1 Jan 30 02:41 /dev/nvidia-uvm-tools


위의 nvidia0 및 nvidiactrl 등을 통하여 nvidia GPU 장치가 설치되어 있음을 알 수 있습니다.

자 그럼 장치와 컴파일러가 잘 설치되어 있음을 확인하였으니, nvcc 명령어를 이용하여 CUDA 코드를 컴파일을 해보도록 하겠습니다.

명령어의 가장 단순한 컴파일 형식은 다음과 같습니다.
   * nvcc cuda_code.cu -o executable_output
   * 실예] nvcc ./1hello.cu -o 1hello
   

In [0]:
!nvcc ./1hello.cu

이후, 컴파일된 실행화일인 a.out을 실행시키면 해당 결과가 프린트 됩니다.

In [16]:
!./a.out

Hello thread 0 in block 2
Hello thread 1 in block 2
Hello thread 2 in block 2
Hello thread 3 in block 2
Hello thread 0 in block 1
Hello thread 1 in block 1
Hello thread 2 in block 1
Hello thread 3 in block 1
Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0


위의 코드를 수정하고 싶다면 어떻게 하면 좋을까요 ?
" %%writefile hello_cuda.cu "를 이용해서 진행하면 좋을 것 같습니다.


In [17]:
%%writefile hello_cuda.cu
# include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  helloCUDA<<<2, 3>>>();
  cudaDeviceReset();
  return 0;
}


Writing hello_cuda.cu


In [0]:
!nvcc ./hello_cuda.cu -o hello_cuda

In [19]:
!ls

00_reduction_all.cu		   07_matmul_tile4_unroll.cu  clock.cu
00_reduction_final.cu		   1hello.cu		      CUDA_lab.txt
01_reduction.cu			   2Dindex.cu		      globalMemory.cu
02_reduction.cu			   2index.cu		      hello_cuda
03_matmul.cu			   a.out		      hello_cuda.cu
04_matmul_tile.cu		   async_streams.cu	      README.md
05_matmul_tile2_mem_coel.cu	   atomicAdd.cu		      transpose.cu
06_matmul_tile3_noBankConflict.cu  bankConflict.cu


In [20]:
!./a.out

Hello thread 0 in block 2
Hello thread 1 in block 2
Hello thread 2 in block 2
Hello thread 3 in block 2
Hello thread 0 in block 1
Hello thread 1 in block 1
Hello thread 2 in block 1
Hello thread 3 in block 1
Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0


오늘 colaboratory를 활용하여 간단히 GPU CUDA Programming을 하는 방법을 보여드렸습니다. 잘 활용하시면 좋을 것 같네요!
감사합니다.
